In [60]:
import pandas as pd
import spacy 
nlp = spacy.load('en')
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter 
from bs4 import BeautifulSoup 
import nltk
from nltk.corpus import sentiwordnet as swn
import re
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer("english")
stop = set(stopwords.words('english'))

import pickle

def dump(obj,filename):
    filehandler = open(filename,"wb")
    pickle.dump(obj,filehandler)
    filehandler.close()

def load(filename):
    file = open(filename,'rb')
    obj = pickle.load(file)
    file.close()
    return obj

In [3]:
# CLEANS TEXT BY REPLACING CHARACTERS
def getCleanText(text, lower = True):
    text = str(text)
    parse_text = BeautifulSoup(text).get_text()
    letters_only = re.sub(r'http[s]?:\/\/(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' url ', 
                          parse_text, flags=re.MULTILINE)
    letters_only = re.sub('^(1?[0-9]|2[0-3]):[0-5][0-9]$','time',letters_only)
    letters_only = re.sub('(name|NAME)\d{1,}','username',letters_only)
    letters_only = re.sub("[^a-zA-Z0-9\.:']",  
                      " ",                   
                      letters_only)
    letters_only = letters_only.replace('\n'," ")
    letters_only = letters_only.replace('\r'," ")
    letters_only = re.sub('[.]{2,}', '. ', letters_only)
    if lower:
        letters_only = letters_only.lower()
    #words = word_tokenize(lower)
    words = CountVectorizer(stop_words='english').build_tokenizer()(letters_only)
    meaningful_words = [ w for w in words if len(w)> 0 and len(w)<20]
    #meaningful_words = [w if (w not in countries) and (w not in cities) else u"country" for w in meaningful_words]
    #lem_words = [wordnet_lemmatizer.lemmatize(w) for w in  meaningful_words]
    #stem_words = [snowball_stemmer.stem(w) for w in lem_words]
    clean_text = " ".join(meaningful_words)
    return clean_text

In [61]:
# LOAD DF FROM FILE
conversations = pd.read_excel('Coke vs Pepsi Twitter Data for NLP 1.xlsx')
conversations = conversations["Sound Bite Text"]

In [64]:
# OR LOAD LIST FROM DATA
conversations = load("../corpus_doc.pkl")

In [86]:
def scoring(doc):
    stokens = [nltk.word_tokenize(doc)]
    taggedlist=[]
    for stoken in stokens:        
        taggedlist.append(nltk.pos_tag(stoken))
    wnl = nltk.WordNetLemmatizer()
    score_list=[]
    for idx,taggedsent in enumerate(taggedlist):
        score_list.append([])
        for idx2,t in enumerate(taggedsent):
            newtag=''
            lemmatized=wnl.lemmatize(t[0])
            if t[1].startswith('NN'):
                newtag='n'
            elif t[1].startswith('JJ'):
                newtag='a'
            elif t[1].startswith('V'):
                newtag='v'
            elif t[1].startswith('R'):
                newtag='r'
            else:
                newtag=''       
            if(newtag!=''):    
                synsets = list(swn.senti_synsets(lemmatized, newtag))
                score=0
                if(len(synsets)>0):
                    for syn in synsets:
                        score+=syn.pos_score()-syn.neg_score()
                    score_list[idx].append(score/len(synsets))
    sentence_sentiment=[]
    # get the average score per word
    for score_sent in score_list:
        if len(score_sent) != 0:
            sentence_sentiment.append(sum([word_score for word_score in score_sent])/len(score_sent))
        else:
            continue
#    print("Sentiment for each sentence for:"+doc)
    return sentence_sentiment

In [87]:
# standarize text for tagging and sentiment scoring
clean_conversations = map(getCleanText, conversations)

In [88]:
sentiment_scores = map(scoring, clean_conversations)
sentence_sentiment = list(sentiment_scores)

/Users/erikdaily/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/erikdaily/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
